<a href="https://colab.research.google.com/github/minofeel/-/blob/main/binary_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import pandas as pd
# from tqdm.notebook import tqdm
from time import time
import math
from pytz import timezone
import argparse
import random
import sklearn
# import torch.fx
# from ipywidgets import IntProgress
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import tqdm
import copy
from datetime import datetime
from sklearn.metrics import classification_report
import glob
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
import torchvision.transforms as transforms

import torch.optim as optim

from pytz import timezone
import datetime as dt

from torch.utils.data import DataLoader

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

In [ ]:
def flatten(xss):
    return [x for xs in xss for x in xs]

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_transforms = A.Compose(
    [
        A.Resize(height=512, width=512),
        A.Rotate(p=0.1, limit=(-10,10)),
        A.RandomBrightness(p=0.1, limit=(-0.1, 0.1)),
        A.Normalize(mean=(0.44, 0.44, 0.44), std=(0.2447, 0.2447, 0.2447)),
        ToTensorV2(),
    ]
)

test_transforms = A.Compose(
    [
        A.Resize(height=512, width=512),
        A.Normalize(mean=(0.44, 0.44, 0.44), std=(0.2447, 0.2447, 0.2447)),
        ToTensorV2(),
    ]
)

/opt/conda/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [ ]:
####################################################
#       Create Train, Valid and Test sets
####################################################
train_data_path = '/workspace/fake_data/Pneumothorax_fake_train'
test_data_path = '/workspace/fake_data/Pneumothorax_fake_test'

train_image_paths = []
classes = []

for data_path in glob.glob(train_data_path + '/*'):
    classes.append(data_path.split('/')[-1])
    train_image_paths.append(glob.glob(data_path + '/*'))

train_image_paths = list(flatten(train_image_paths))
random.shuffle(train_image_paths)

print('train_image_path example: ', train_image_paths[0])
print('class example:', classes)

#2.
# split train valid from train paths (80,20)
train_image_paths, valid_image_paths = train_image_paths[:int(0.8889*len(train_image_paths))], train_image_paths[int(0.8889*len(train_image_paths)):]
# valid_image_paths, test_image_paths = valid_image_paths[:int(0.9*len(valid_image_paths))], valid_image_paths[int(0.1*len(valid_image_paths)):]

# 3.
# create the test_image_paths
test_image_paths = []
for data_path in glob.glob(test_data_path + '/*'):
    test_image_paths.append(glob.glob(data_path + '/*'))

test_image_paths = list(flatten(test_image_paths))

print("Train size: {}\nValid size: {}\nTest size: {}".format(len(train_image_paths), len(valid_image_paths
                                                                                        ), len(test_image_paths)))


train_image_path example:  /workspace/fake_data/Pneumothorax_fake_train/pneumothorax/ChestCT_Pneumothorax_00001001.png
class example: ['normal', 'pneumothorax']
Train size: 2000
Valid size: 250
Test size: 250


In [ ]:
idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}
class_to_idx

{'normal': 0, 'pneumothorax': 1}

In [ ]:
class DiseaseDataset(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label = image_filepath.split('/')[-2]
        label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(image=image)["image"]

        return image, label

#######################################################
#                  Create Dataset
#######################################################

train_dataset = DiseaseDataset(train_image_paths,train_transforms)
valid_dataset = DiseaseDataset(valid_image_paths,test_transforms) #test transforms are applied
test_dataset = DiseaseDataset(test_image_paths,test_transforms)

In [ ]:
print('The shape of tensor for 50th image in train dataset: ',train_dataset[46][0].shape)
print('The label for 50th image in train dataset: ',train_dataset[0][1])

The shape of tensor for 50th image in train dataset:  torch.Size([3, 512, 512])
The label for 50th image in train dataset:  1


In [ ]:
#######################################################
#                  Define Dataloaders
#######################################################

train_loader = DataLoader(
    train_dataset, batch_size=32, shuffle=True, drop_last=True,
)

valid_loader = DataLoader(
    valid_dataset, batch_size=32, shuffle=True, drop_last= True,
)


test_loader = DataLoader(
    test_dataset, batch_size=1, shuffle=False,
)

In [ ]:
# loader = {'train':train_loader, 'val':valid_loader, 'test':test_loader}
loader = {'train':train_loader, 'val':valid_loader, 'test':test_loader}

In [ ]:
next(iter(train_loader))[0].shape

torch.Size([32, 3, 512, 512])

In [ ]:
from torchvision import models

model = models.resnet101(pretrained=True)

# model.fc = nn.Linear(2048,16)
# model.relu = nn.Tanh()
# # model.layer1.relu = nn.Tanh()
# model.layer1[0].relu = nn.Tanh()
# model.layer1[1].relu = nn.Tanh()
# model.layer1[2].relu = nn.Tanh()
# model.layer2[0].relu = nn.Tanh()
# model.layer2[1].relu = nn.Tanh()
# model.layer2[2].relu = nn.Tanh()
# model.layer2[3].relu = nn.Tanh()


# model.layer3[0].relu = nn.Tanh()
# model.layer3[1].relu = nn.Tanh()
# model.layer3[2].relu = nn.Tanh()
# model.layer3[3].relu = nn.Tanh()
# model.layer3[4].relu = nn.Tanh()
# model.layer3[5].relu = nn.Tanh()


# model.layer4[0].relu = nn.Tanh()
# model.layer4[1].relu = nn.Tanh()
# model.layer4[2].relu = nn.Tanh()


model.fc.out_features = 2
model.to(device)


model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
best_acc = 0.0
batch_size = 256
# num_epochs = 10
for epoch in range(60):
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(train_loader):
        # print(data)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs,labels)

        # loss = criterion(outputs, labels)
        # print(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 100 == 99:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

    for i, data in enumerate(valid_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            # loss = criterion(outputs, labels)
            loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # print_roc_auc(y_score=y_score, y_test=y_test)
    # print("[val] epoch {} loss: {:.4f}".format(epoch + 1, loss.item()))
    print(f'[{epoch} epoch] Accuracy of the network on the validation images: {100* correct/total}')
    # if best_acc < (100*correct/total):
        # torch.save(model.state_dict(), f'./{epoch}_best.pt')
    best_acc = (100 * correct/total)
    torch.save(model, f'./classifier/real_model/LungCancer/{epoch}+{best_acc}+{i}.pth')

[0 epoch] Accuracy of the network on the validation images: 81.47644927536231
[1 epoch] Accuracy of the network on the validation images: 95.15398550724638
[2 epoch] Accuracy of the network on the validation images: 97.69021739130434
[3 epoch] Accuracy of the network on the validation images: 98.0072463768116
[4 epoch] Accuracy of the network on the validation images: 98.27898550724638
[5 epoch] Accuracy of the network on the validation images: 97.14673913043478
[6 epoch] Accuracy of the network on the validation images: 97.5090579710145
[7 epoch] Accuracy of the network on the validation images: 98.91304347826087
[8 epoch] Accuracy of the network on the validation images: 98.91304347826087
[9 epoch] Accuracy of the network on the validation images: 99.32065217391305
[10 epoch] Accuracy of the network on the validation images: 98.05253623188406
[11 epoch] Accuracy of the network on the validation images: 98.41485507246377
[12 epoch] Accuracy of the network on the validation images: 99.

KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
def plt_roc(test_y, probas_y, plot_micro=False, plot_macro=False):
    assert isinstance(test_y, list) and isinstance(probas_y, list), 'the type of input must be list'

    y = label_binarize(test_y, classes=[0, 1, 2, 3])
    probas_y = np.array(probas_y)
    n_classes = ['normal', 'cancer']

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    # compute fpr and tpr with roc_curve from the ytest true labels to the scores
    for i in range(len(n_classes)):
        fpr[i], tpr[i], _ = roc_curve(y[:, i], np.array(probas_y)[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), np.array(probas_y).ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    plt.figure(figsize=(8,8))

    plt.plot(fpr["micro"], tpr["micro"], color='black',
             label='micro-average ROC curve (area = {0:0.4f})'
                   ''.format(roc_auc["micro"]))

    colors = ['blue', 'red', 'green', 'pink', 'yellow']
    for i, color, lbl in zip(range(len(n_classes)), colors, n_classes):
        plt.plot(fpr[i], tpr[i], color = color, lw = 1.5,
        label = 'ROC Curve of class {0} (area = {1:0.4f})'.format(lbl, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw = 1.5)
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Real Chest CT Image')
    plt.legend(loc = 'lower right', prop = {'size': 8})
    plt.savefig('roc_auc_curve.png')
    plt.close()

In [ ]:
import time
import statistics

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

model = torch.load('classifier/fake_model/Pnuemothorax/2+99.72826086956522+6.pth')
model.to(device)
model.eval()

y = []
y_pred = []
probas_y = []
start = time.time()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probas_y.append(outputs[0][0:4].data.cpu().numpy().tolist())
        _, predictions = torch.max(outputs, 1)

        for label, prediction in zip(labels, predictions):
            y.append(int(label))
            y_pred.append(int(prediction))
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

plt_roc(y, probas_y)

correct_list = []
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.3f} %')
    correct_list.append(accuracy)

correct_mean = statistics.mean(correct_list)
print(f'Accuracy for all class: {correct_mean} %')

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

Accuracy for class: normal is 100.000 %
Accuracy for class: pneumothorax is 100.000 %
Accuracy for all class: 100.0 %
time : 6.980006456375122
